# Generando Datos Aleatorios con Python para SQL Server

## Librerias:

- Faker
- Pandas
- Random
- DateTime
- PyODBC
- fast\_to\_sql
    

| Libreria | Uso |
| --- | --- |
| Faker | Esta libreria nos permite generar datos aleatorios para Nombres, Apellidos, Ciudades, ZipCodes, Booleanos, entre otros. Nos da la flexibilidad de especificar una cultura o lenguaje y nos facilita la creación de datos únicos |
| Pandas | Aqui manejaremos nuestros datos, los organizaremos y moldearemos para luego llevarlos a SQL Server |
| Random | El nombre lo dice todo, esta libreria nos permite generar digitos aleatorios |
| DateTime | Con esta libreria manejaremos fechas |
| PyODBC | Maneja conexiones ODBC |
|fast_to_sql|Permite hacer inserciones rapidas a SQL Server||

# Comencemos...

## Requerimientos

Necesitamos generar al menos 10k registros para una tabla con las siguientes especificaciones:

| Columna | Tipo | Restricciones/Especificaciones |
| --- | --- | --- |
| ID | int | Sequencial |
| Nombre | String |  |
| Apellido | String |  |
| Direccion | String |  |
| Ciudad | String |  |
| Telefono | String |  |
| CorreoElectronico | String | Unique |
| Edad | Int | Limitado de 18-65 |
| GrupoID | Int | Limitado de 1-4 |
| Activo | Bool |  |
| IDUniversal | GUID | Unique |
| BalanceNeto | Float | $500-$10k |
| FechaBalance | Date | Entre Mar-2019 y Nov 2020 |

## Primero Imports...

In [ ]:
from faker import Faker
import pandas as pd
import random
from datetime import datetime
import pyodbc
from fast_to_sql import fast_to_sql as fts

## Definamos el lenguaje de Faker e inicializemos la clase y ciertas variables para la corrida.

In [ ]:
fake = Faker(['en-US'])
Start = 1
End = 10001
StartDate = '03-01-19'
EndDate = '11-30-20'

---

## En muchos casos, he visto este método ser utilizado para la creacion de registros basados en calculos. Este método es recursivo e ineficiente. 

In [ ]:
FrameRecursive

In [ ]:
FrameRecursive = pd.DataFrame()
Counter = 1
rbegin = datetime.now()
while Counter < End:
    Nombre = fake.first_name()
    Apellido = fake.last_name()
    Direccion = fake.address()
    Ciudad = fake.city()
    Telefono = fake.phone_number()
    proveedor = fake.free_email_domain()
    CorreoElectronico = Nombre+"."+Apellido+"@"+proveedor
    Edad = random.randint(18, 65)
    GrupoID = random.randint(1, 4)
    Activo = fake.boolean(chance_of_getting_true=75)
    IDUniversal = fake.unique.uuid4()
    BalanceNeto = float("{0:.2f}".format(random.uniform(500, 10000)))
    SD = datetime.strptime(StartDate, '%m-%d-%y')
    ED = datetime.strptime(EndDate, '%m-%d-%y')
    FechaBalance = str(fake.date_between(
            start_date=SD, end_date=ED))
    FrameRecursive = FrameRecursive.append({  'ID':Counter,
                                            'Nombre':Nombre,
                                            'Apellido':Apellido,
                                            'Direccion':Direccion,
                                            'Ciudad':Ciudad,
                                            'Telefono':Telefono,
                                            'CorreoElectronico':CorreoElectronico,
                                            'Edad':Edad,
                                            'GrupoID':GrupoID,
                                            'Activo':Activo,
                                            'IDUniversal':IDUniversal,
                                            'BalanceNeto':BalanceNeto,
                                            'FechaBalnce':FechaBalance
                                         },ignore_index=True)
    Counter =Counter + 1
    rend=datetime.now()
    rDiff = (rend-rbegin).total_seconds()

## LAMBDAS como metodo de inserción.

Al utilizar lambdas, la serie del panda es evaluada en linea y nos permite generar los datos aleatorios sin necesidad de iterar linea por linea utilizadon un:

```
for k in GeneratedData.iterrows()
```

El método mencionado arriba, generaria los registros uno a uno, lo que puede consumir tiempo y memoria al generar miles de registros. Sin embargo al utlizar funciones atravez de <strong>lambas</strong> se generan a nivel de pivote en el panda lo que resulta en mejor rendimiento y codigo mucho mas limpio. Para comenzar el Dataframe insertaremos valores en la columna ID secuencialmente basados en el rango definido anteriormente.

### Primero: Vamos a crear funciones para cada columna, esto nos permitira evitar el uso de FOR o WHILE loops y hacer nuestro codigo mas legible.

In [ ]:
def GenNombre():
    Nombre = fake.first_name()
    return Nombre

def GenApellido():
    Apellido = fake.last_name()
    return Apellido 

def GenDireccion():
    Direccion = fake.address()
    return Direccion 

def GenCiudad():
    Ciudad = fake.city()
    return Ciudad

def GenTelefono():
    Telefono = fake.phone_number()
    return Telefono

def GenCorreoElectronico(Nombre,Apellido):
    proveedor = fake.free_email_domain()
    CorreoElectronico = Nombre+"."+Apellido+"@"+proveedor
    return CorreoElectronico

def GenEdad():
    Edad = random.randint(18, 65)
    return Edad

def GenGrupoID():
    GrupoID = random.randint(1, 4)
    return GrupoID

def GenActivo():
    Activo = fake.boolean(chance_of_getting_true=75)
    return Activo

def GenIDUniversal():
    IDUniversal = fake.unique.uuid4()
    return IDUniversal

def GenBalanceNeto():
    BalanceNeto = float("{0:.2f}".format(random.uniform(500, 10000)))
    return BalanceNeto

def GenFechaBalance():
    SD = datetime.strptime(StartDate, '%m-%d-%y')
    ED = datetime.strptime(EndDate, '%m-%d-%y')
    FechaBalance = str(fake.date_between(
            start_date=SD, end_date=ED))
    return FechaBalance

## Crearemos el DataFrame y lo popularemos utilizando Lambdas de la siguiente manera.

In [ ]:
begin = datetime.now()
GeneratedData = pd.DataFrame(list(range(Start,End)), columns=["ID"])
GeneratedData["Nombre"] = GeneratedData.apply(lambda x: GenNombre(), axis=1)
GeneratedData["Apellido"] = GeneratedData.apply(lambda x: GenApellido(), axis=1)
GeneratedData["Direccion"] = GeneratedData.apply(lambda x: GenDireccion(), axis=1)
GeneratedData["Ciudad"] = GeneratedData.apply(lambda x: GenCiudad(), axis=1)
GeneratedData["Telefono"] = GeneratedData.apply(lambda x: GenTelefono(), axis=1)
GeneratedData["CorreoElectronico"] = GeneratedData.apply(lambda x: GenCorreoElectronico(x.Nombre,x.Apellido), axis=1)
GeneratedData["Edad"] = GeneratedData.apply(lambda x: GenEdad(), axis=1)
GeneratedData["GrupoID"] = GeneratedData.apply(lambda x: GenGrupoID(), axis=1)
GeneratedData["Activo"] = GeneratedData.apply(lambda x: GenActivo(), axis=1)
GeneratedData["IDUniversal"] = GeneratedData.apply(lambda x: GenIDUniversal(), axis=1)
GeneratedData["BalanceNeto"] = GeneratedData.apply(lambda x: GenBalanceNeto(), axis=1)
GeneratedData["FechaBalance"] = GeneratedData.apply(lambda x: GenFechaBalance(), axis=1)
end = datetime.now()
diff = (end-begin).total_seconds()

In [ ]:
GeneratedData

## Comparemos los Resultados 

In [41]:
RegistrosCount = GeneratedData["ID"].count()
RecCount = FrameRecursive["ID"].count()
print("Recursive approach inserted {0} rows and took {1} seconds to complete".format(RecCount, rDiff))
print("Lambda approach inserted {0} rows and took {1} seconds to complete".format(RegistrosCount, diff))


Recursive approach inserted 10000 rows and took 117.846234 seconds to complete
Lambda approach inserted 10000 rows and took 10.197089 seconds to complete


## Panda -\> SQL

Comenzemos por definir e inicializar la conexión a nuestro servidor de SQL utilizando PyODBC y FastToSQL

In [42]:
def ConnectToSQL():
    server = "localhost,1500"
    database ="PythonGenerated"
    username = "sa"
    password = "Today123"
    conn = pyodbc.connect(
        "DRIVER={{ODBC Driver 17 for SQL Server}};SERVER={0}; database={1}; trusted_connection=no;UID={2};PWD={3}".format(
            server, database, username, password
        ))

    return conn

## Enviar nuestros datos a SQL utilizando TO_JSON (parte de pandas)

In [43]:
Context = ConnectToSQL()
create_statement = fts.fast_to_sql(GeneratedData, "Aleatorios", Context, if_exists="replace", temp=False)
Context.commit()
Context.close()